In [ ]:
import os
import subprocess
import re

def time_to_seconds(time_str):
    """ 시:분:초 또는 분:초 형식의 문자열을 초 단위로 변환 """
    # 시간 형식 확인을 위한 정규 표현식 패턴
    pattern = r'^(?:(?:(\d+):)?(\d+):)?(\d+)$'
    match = re.match(pattern, time_str.strip())
    
    if not match:
        raise ValueError("올바른 시간 형식이 아닙니다. '시:분:초', '분:초' 또는 '초' 형식으로 입력하세요.")
    
    hours, minutes, seconds = match.groups()
    
    # None 값을 0으로 변환
    hours = int(hours) if hours else 0
    minutes = int(minutes) if minutes else 0
    seconds = int(seconds)
    
    # 초 단위로 변환
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

def download_youtube_clip(url, start_time, end_time, output_filename=None):
    """ 유튜브 URL에서 특정 시간대의 클립 다운로드 """
    temp_filename = "temp_video.mp4"
    
    try:
        # 문자열 형식의 시간을 초 단위로 변환
        if isinstance(start_time, str):
            start_time = time_to_seconds(start_time)
        if isinstance(end_time, str):
            end_time = time_to_seconds(end_time)
            
        # 유튜브 영상 다운로드
        print("영상 다운로드 중...")
        download_cmd = [
            "yt-dlp", 
            "--cookies-from-browser", "chrome", 
            "-f", "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best", 
            "-o", temp_filename,
            url
        ]
        subprocess.run(download_cmd, check=True)
        
        # 출력 파일 이름 설정
        if output_filename is None:
            output_filename = "output.mp4"
        else:
            output_filename = f"{output_filename}.mp4"
        
        # 클립 추출
        print(f"클립 추출 중: {start_time}초 - {end_time}초...")
        
        duration = end_time - start_time
        ffmpeg_cmd = [
            "ffmpeg",
            "-i", temp_filename,
            "-ss", str(start_time),
            "-t", str(duration),
            "-c:v", "libx264",
            "-c:a", "aac",
            "-strict", "experimental",
            output_filename
        ]
        subprocess.run(ffmpeg_cmd, check=True)
        
        # 임시 파일 삭제
        if os.path.exists(temp_filename):
            os.remove(temp_filename)
            print(f"임시 파일 삭제 완료")
        
        print(f"클립 저장 완료: {output_filename}")
        return os.path.abspath(output_filename)
    
    except Exception as e:
        import traceback
        print(f"오류 발생: {str(e)}")
        print(traceback.format_exc())  # 자세한 오류 정보 출력
        
        # 임시 파일 정리
        if os.path.exists(temp_filename):
            os.remove(temp_filename)
        return None

if __name__ == "__main__":
    video_url = input("유튜브 URL을 입력하세요: ")
    
    # 시간 입력 안내 메시지 개선
    start_time_str = input("클립 시작 시간을 입력하세요: ")
    end_time_str = input("클립 종료 시간을 입력하세요: ")
    
    output_name = input("출력 파일 이름을 입력하세요: ") or None
    
    clip_path = download_youtube_clip(video_url, start_time_str, end_time_str, output_name)
    
    if clip_path:
        print(f"클립이 다음 위치에 저장되었습니다: {clip_path}")
    else:
        print("클립 다운로드에 실패했습니다.")

영상 다운로드 중...
Extracting cookies from chrome
Extracted 453 cookies from chrome
[youtube] Extracting URL: https://youtu.be/n_f5mVyG7y4
[youtube] n_f5mVyG7y4: Downloading webpage
[youtube] n_f5mVyG7y4: Downloading tv client config
[youtube] n_f5mVyG7y4: Downloading player 14cfd4c0-main
[youtube] n_f5mVyG7y4: Downloading tv player API JSON
[info] n_f5mVyG7y4: Downloading 1 format(s): 399+140
[download] Destination: temp_video.f399.mp4
[download] 100% of  171.52MiB in 00:00:17 at 9.55MiB/s     
[download] Destination: temp_video.f140.m4a
[download] 100% of   28.10MiB in 00:00:03 at 7.36MiB/s     
[Merger] Merging formats into "temp_video.mp4"
Deleting original file temp_video.f399.mp4 (pass -k to keep)
Deleting original file temp_video.f140.m4a (pass -k to keep)
클립 추출 중: 1065초 - 1068초...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspee

임시 파일 삭제 완료
클립 저장 완료: cheese_in_the_trap_test.mp4
클립이 다음 위치에 저장되었습니다: /Users/leenayoung/Desktop/SRT/SRT_SignalProcessing/cheese_in_the_trap_test.mp4


[out#0/mp4 @ 0x141804110] video:848KiB audio:47KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.518197%
frame=   89 fps=3.9 q=-1.0 Lsize=     900KiB time=00:00:02.93 bitrate=2510.9kbits/s speed=0.13x    
[libx264 @ 0x1418051b0] frame I:2     Avg QP:20.07  size: 58992
[libx264 @ 0x1418051b0] frame P:24    Avg QP:21.97  size: 19756
[libx264 @ 0x1418051b0] frame B:63    Avg QP:24.67  size:  4377
[libx264 @ 0x1418051b0] consecutive B-frames:  4.5%  2.2%  3.4% 89.9%
[libx264 @ 0x1418051b0] mb I  I16..4: 21.8% 69.7%  8.5%
[libx264 @ 0x1418051b0] mb P  I16..4:  5.1% 10.4%  0.4%  P16..4: 33.9%  7.0%  2.3%  0.0%  0.0%    skip:40.9%
[libx264 @ 0x1418051b0] mb B  I16..4:  0.5%  0.6%  0.0%  B16..8: 29.6%  0.9%  0.1%  direct: 0.7%  skip:67.5%  L0:44.9% L1:52.5% BI: 2.6%
[libx264 @ 0x1418051b0] 8x8 transform intra:65.0% inter:90.4%
[libx264 @ 0x1418051b0] coded y,uvDC,uvAC intra: 33.4% 39.3% 5.2% inter: 4.7% 5.8% 0.0%
[libx264 @ 0x1418051b0] i16 v,h,dc,p: 27% 30%  4% 38%
[